In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fpdf import FPDF
from docx import Document
import os
import webbrowser
import platform

def get_download_path(filename):
    if platform.system() == "Windows":
        path = os.path.join(os.environ['USERPROFILE'], 'Downloads', filename)
    else:
        path = os.path.join(os.path.expanduser('~'), 'Downloads', filename)
    return path

def generate_data_report(dataframe):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    missing_values = dataframe.isnull().sum()[dataframe.isnull().sum() > 0]

    numeric_cols = dataframe.select_dtypes(include=['number']).columns.tolist()
    non_numeric_cols = dataframe.select_dtypes(exclude=['number']).columns.tolist()
    data_types = dataframe.dtypes.to_dict()

    duplicates_before = []
    columns_seen = set()
    for col1 in dataframe.columns:
        for col2 in dataframe.columns:
            if col1 != col2 and dataframe[col1].equals(dataframe[col2]) and col2 not in duplicates_before:
                duplicates_before.append(col2)
                columns_seen.add(col1)

    columns_to_drop = []
    for col1 in dataframe.columns:
        for col2 in dataframe.columns:
            if col1 != col2 and dataframe[col1].equals(dataframe[col2]) and col2 not in columns_to_drop and col1 in columns_seen:
                columns_to_drop.append(col2)
    
    dataframe.drop(columns=columns_to_drop, inplace=True)
    
    duplicates_after = []
    for col1 in dataframe.columns:
        for col2 in dataframe.columns:
            if col1 != col2 and dataframe[col1].equals(dataframe[col2]) and col2 not in duplicates_after:
                duplicates_after.append(col2)

    constant_columns_before = [col for col in dataframe.columns if dataframe[col].nunique() == 1]
    dataframe.drop(columns=constant_columns_before, inplace=True)
    constant_columns_after = [col for col in dataframe.columns if dataframe[col].nunique() == 1]

    create_boxplots(dataframe, get_download_path('boxplots'))
    create_distribution_charts(dataframe, get_download_path('distributions'), dataframe.columns[:6])

    html_path = export_to_html(dataframe, missing_values, numeric_cols, non_numeric_cols, data_types, duplicates_before, duplicates_after, constant_columns_before, constant_columns_after)
    pdf_path = export_to_pdf(missing_values, numeric_cols, non_numeric_cols, data_types, duplicates_before, duplicates_after, constant_columns_before, constant_columns_after)
    word_path = export_to_word(missing_values, numeric_cols, non_numeric_cols, data_types, duplicates_before, duplicates_after, constant_columns_before, constant_columns_after)

    webbrowser.open(html_path)
    os.system(f'open "{pdf_path}"') if platform.system() == "Darwin" else os.startfile(pdf_path)
    os.system(f'open "{word_path}"') if platform.system() == "Darwin" else os.startfile(word_path)

    print(f"Reports saved and opened: {html_path}, {pdf_path}, {word_path}")

def create_boxplots(df, folder):
    os.makedirs(folder, exist_ok=True)
    numeric_cols = df.select_dtypes(include=['number']).columns
    for col in numeric_cols:
        plt.figure(figsize=(8, 6))
        sns.boxplot(df[col], color="skyblue")
        plt.title(f"Box Plot - {col}")
        plt.xlabel(col)
        plt.savefig(os.path.join(folder, f"{col}_boxplot.png"), bbox_inches="tight")
        plt.close()

def create_distribution_charts(df, folder, columns):
    os.makedirs(folder, exist_ok=True)
    for col in columns:
        plt.figure(figsize=(8, 6))
        if df[col].dtype in ['int64', 'float64']:
            sns.histplot(df[col], kde=True, color="green", bins=20)
        else:
            sns.countplot(y=df[col], palette="viridis")
        plt.title(f"Distribution - {col}")
        plt.savefig(os.path.join(folder, f"{col}_distribution.png"), bbox_inches="tight")
        plt.close()

def export_to_html(df, missing_values, numeric_cols, non_numeric_cols, data_types, duplicates_before, duplicates_after, constant_columns_before, constant_columns_after):
    html_path = get_download_path("report.html")

    duplicates_before_display = f"[{', '.join(duplicates_before)}]" if duplicates_before else "[]"
    duplicates_after_display = f"[{', '.join(duplicates_after)}]" if duplicates_after else "[]"
    constant_columns_before_display = f"[{', '.join(constant_columns_before)}]" if constant_columns_before else "[]"
    constant_columns_after_display = f"[{', '.join(constant_columns_after)}]" if constant_columns_after else "[]"

    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Data Report</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 20px;
                background-color: #f9f9f9;
                color: #333;
            }}
            h1, h2 {{
                color: #2c3e50;
            }}
            .container {{
                max-width: 900px;
                margin: auto;
                background: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
            }}
            ul {{
                list-style: disc;
                padding-left: 20px;
            }}
            table {{
                border-collapse: collapse;
                width: 100%;
                margin: 10px 0;
            }}
            th, td {{
                border: 1px solid #ddd;
                padding: 10px;
                text-align: left;
            }}
            th {{
                background-color: #f4f4f4;
            }}
            .section {{
                margin-bottom: 30px;
            }}
            .section h2 {{
                border-bottom: 2px solid #2c3e50;
                padding-bottom: 5px;
                margin-bottom: 15px;
            }}
            .visualization-links ul {{
                padding-left: 20px;
            }}
            a {{
                color: #3498db;
                text-decoration: none;
            }}
            a:hover {{
                text-decoration: underline;
            }}
            pre {{
                white-space: pre-wrap;
                word-wrap: break-word;
                max-width: 100%;
                overflow-wrap: break-word;
                word-break: break-all;
                line-height: 1.5;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Data Analysis Report</h1>
            
            <div class="section">
                <h2>Missing Values</h2>
                <table>
                    <thead>
                        <tr>
                            <th>Column Name</th>
                            <th>Missing Count</th>
                        </tr>
                    </thead>
                    <tbody>
                        {''.join(f"<tr><td>{col}</td><td>{val}</td></tr>" for col, val in missing_values.items())}
                    </tbody>
                </table>
            </div>

            <div class="section">
                <h2>Numeric Columns</h2>
                <ul>
                    {"".join(f"<li>{col}</li>" for col in numeric_cols)}
                </ul>
            </div>

            <div class="section">
                <h2>Non-Numeric Columns</h2>
                <ul>
                    {"".join(f"<li>{col}</li>" for col in non_numeric_cols)}
                </ul>
            </div>

            <div class="section">
                <h2>Data Types</h2>
                <ul>
                    {"".join(f"<li>{col}: {dtype}</li>" for col, dtype in data_types.items())}
                </ul>
            </div>

            <div class="section">
                <h2>Duplicate Columns</h2>
                <h3>Before Removal</h3>
                <pre>{duplicates_before_display}</pre>
                <h3>After Removal</h3>
                <pre>{duplicates_after_display}</pre>
            </div>

            <div class="section">
                <h2>Constant Columns</h2>
                <h3>Before Removal</h3>
                <pre>{constant_columns_before_display}</pre>
                <h3>After Removal</h3>
                <pre>{constant_columns_after_display}</pre>
            </div>

            <div class="section">
                <h2>Visualizations</h2>
                <div class="visualization-links">
                    <h3>Box Plots</h3>
                    <ul>
                        {''.join(f'<li><a href="file://{get_download_path("boxplots")}/{col}_boxplot.png">Box plot of {col}</a></li>' for col in numeric_cols)}
                    </ul>

                    <h3>Distribution Charts</h3>
                    <ul>
                        {''.join(f'<li><a href="file://{get_download_path("distributions")}/{col}_distribution.png">Distribution of {col}</a></li>' for col in df.columns[:6])}
                    </ul>
                </div>
            </div>
        </div>
    </body>
    </html>
    """
    with open(html_path, 'w') as file:
        file.write(html_content)

    return html_path

def export_to_pdf(missing_values, numeric_cols, non_numeric_cols, data_types, duplicates_before, duplicates_after, constant_columns_before, constant_columns_after):
    pdf_path = get_download_path("report.pdf")

    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    
    pdf.set_font("Arial", size=12)
    
    pdf.cell(200, 10, txt="Data Analysis Report", ln=True, align='C')

    pdf.ln(10)
    pdf.cell(200, 10, txt="Missing Values", ln=True)
    for col, count in missing_values.items():
        pdf.cell(200, 10, txt=f"{col}: {count} missing", ln=True)
    
    pdf.ln(10)
    pdf.cell(200, 10, txt="Numeric Columns", ln=True)
    for col in numeric_cols:
        pdf.cell(200, 10, txt=col, ln=True)

    pdf.ln(10)
    pdf.cell(200, 10, txt="Non-Numeric Columns", ln=True)
    for col in non_numeric_cols:
        pdf.cell(200, 10, txt=col, ln=True)
    
    pdf.ln(10)
    pdf.cell(200, 10, txt="Data Types", ln=True)
    for col, dtype in data_types.items():
        pdf.cell(200, 10, txt=f"{col}: {dtype}", ln=True)
    
    pdf.ln(10)
    pdf.cell(200, 10, txt="Duplicate Columns Before Removal", ln=True)
    pdf.multi_cell(0, 10, txt=f"Before: {duplicates_before if duplicates_before else '[]'}")
    
    pdf.ln(10)
    pdf.cell(200, 10, txt="Duplicate Columns After Removal", ln=True)
    pdf.multi_cell(0, 10, txt=f"After: {duplicates_after if duplicates_after else '[]'}")
    
    pdf.ln(10)
    pdf.cell(200, 10, txt="Constant Columns Before Removal", ln=True)
    pdf.multi_cell(0, 10, txt=f"Before: {constant_columns_before if constant_columns_before else '[]'}")
    
    pdf.ln(10)
    pdf.cell(200, 10, txt="Constant Columns After Removal", ln=True)
    pdf.multi_cell(0, 10, txt=f"After: {constant_columns_after if constant_columns_after else '[]'}")
    
    pdf.output(pdf_path)

    return pdf_path

def export_to_word(missing_values, numeric_cols, non_numeric_cols, data_types, duplicates_before, duplicates_after, constant_columns_before, constant_columns_after):
    word_path = get_download_path("report.docx")

    doc = Document()

    doc.add_heading('Data Analysis Report', 0)
    
    doc.add_heading('Missing Values', level=1)
    for col, count in missing_values.items():
        doc.add_paragraph(f"{col}: {count} missing")

    doc.add_heading('Numeric Columns', level=1)
    for col in numeric_cols:
        doc.add_paragraph(col)

    doc.add_heading('Non-Numeric Columns', level=1)
    for col in non_numeric_cols:
        doc.add_paragraph(col)
    
    doc.add_heading('Data Types', level=1)
    for col, dtype in data_types.items():
        doc.add_paragraph(f"{col}: {dtype}")

    doc.add_heading('Duplicate Columns Before Removal', level=1)
    doc.add_paragraph(f"Before: {duplicates_before if duplicates_before else '[]'}")
    
    doc.add_heading('Duplicate Columns After Removal', level=1)
    doc.add_paragraph(f"After: {duplicates_after if duplicates_after else '[]'}")
    
    doc.add_heading('Constant Columns Before Removal', level=1)
    doc.add_paragraph(f"Before: {constant_columns_before if constant_columns_before else '[]'}")
    
    doc.add_heading('Constant Columns After Removal', level=1)
    doc.add_paragraph(f"After: {constant_columns_after if constant_columns_after else '[]'}")
    
    doc.save(word_path)

    return word_path

def ensure_csv(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.csv':
        print(f"The file is already a CSV: {file_path}")
        return file_path

    elif file_extension == '.xlsx':
        csv_file_path = os.path.splitext(file_path)[0] + ".csv"
        print(f"Converting {file_path} to CSV...")
        
        df = pd.read_excel(file_path)
        df.to_csv(csv_file_path, index=False)
        
        print(f"Converted to: {csv_file_path}")
        return csv_file_path

    else:
        raise ValueError(f"Unsupported file format: {file_extension}. Please provide a .csv or .xlsx file.")

if __name__ == "__main__":
    
    # dataset can be in either csv or xlsx format
    # file_path = 'path/to/your/dataset.csv'
    file_path='/Users/Manu/Downloads/Titanic-Dataset.csv'
           
    try:
        csv_file_path = ensure_csv(file_path)

        df = pd.read_csv(csv_file_path)

        generate_data_report(df)

    except ValueError as e:
        print(e)


The file is already a CSV: /Users/Manu/Downloads/Titanic-Dataset.csv


/var/folders/qh/hffmjnhx6p5f0fs4py7vmdnr0000gp/T/ipykernel_26086/520723908.py:86: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(y=df[col], palette="viridis")
/var/folders/qh/hffmjnhx6p5f0fs4py7vmdnr0000gp/T/ipykernel_26086/520723908.py:86: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(y=df[col], palette="viridis")


Reports saved and opened: /Users/Manu/Downloads/report.html, /Users/Manu/Downloads/report.pdf, /Users/Manu/Downloads/report.docx
